In [25]:
from pathlib import Path
from util import repo_root
import json
from tqdm.notebook import tqdm
from tabulate import tabulate
import pandas as pd
import tiktoken

In [2]:
datadir = repo_root() / 'data' / 'rephrases-10k'
paths = list(datadir.glob('*.jsonl'))
len(paths)

66

In [5]:
path = paths[0]
samples = [json.loads(line) for line in path.open('r')]
len(samples)

82

In [6]:
samples[0]

{'sl_n': 0,
 'num_clauses': 9,
 'nl_statement': 'Let A, B, D, C be points such that trapezoid ABCD is a trapezoid where line AD is equal to line BC. Let E be a point such that E is on the circle circle centered at B with radius BD.',
 'fl_statement': 'A B C D = eq_trapezoid A B C D; E = on_circle E D B',
 'goal_nl': 'BD over DE is identical to DE over BD',
 'goal_fl': 'eqratio B D D E D E B D',
 'rephrase': 'Consider points A, B, C, and D arranged to form trapezoid ABCD, where sides AD and BC are congruent. Place point E on the circle that has B as its center and a radius equal to BD.',
 'md5_item_code': '69f8199b7f51b0b1d3b817b3401845c3'}

In [9]:
enc = tiktoken.encoding_for_model("gpt-4-turbo")

In [23]:
template = repo_root().joinpath('verb', 'rephrase_geo_template.txt').read_text()
tmplnum = len(enc.encode(template.format(nl_statement='')))
tmplnum

92

In [22]:
innum = len(enc.encode(samples[0]['nl_statement'])) + tmplnum
outnum = len(enc.encode(samples[0]['rephrase']))

145

In [30]:
intotal = 0
outtotal = 0
total_samples = 0
for path in tqdm(paths):
	samples = [json.loads(line) for line in path.open('r')]
	total_samples += len(samples)
	intotal += sum(map(len,enc.encode_batch([x['nl_statement'] for x in samples]))) + len(samples) * tmplnum
	outtotal += sum(map(len,enc.encode_batch([x['rephrase'] for x in samples])))
total_samples, intotal, outtotal

  0%|          | 0/66 [00:00<?, ?it/s]

(10000, 1711888, 784832)

In [31]:
intotal * (0.01/1000) + outtotal * (0.03/1000)

40.66384

In [16]:
enc.encode_batch(['hello', 'world and stuff'])

[[15339], [14957, 323, 6392]]

In [12]:


outnum

48

In [32]:
longpath = repo_root().joinpath('data', 'nl_fl_long.csv')
df = pd.read_csv(longpath)
df.head()

,sl_n,num_clauses,nl_statement,fl_statement
0,0,9,OAXS is a quadrangle. G is a points such that ...,O A X S = quadrangle O A X S; G = angle_bisect...
1,1,7,"X, O, and A are defined such that triangle OAX...",O A X = iso_triangle O A X; S = mirror S O A; ...
2,2,8,"Points O, X, S, A are defined such that line O...",O A X S = eqdia_quadrangle O A X S; G = foot G...
3,3,8,O & A are point such that A & O is a segment o...,O A = segment O A; X S G F = trapezoid X S G F...
4,4,5,"Points O, A, X, S, G are defined such that OAX...",O A X S G = pentagon O A X S G; F = on_tline F...


In [33]:
len(df)

1302942